## 1. 데이터 불러오기

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [26]:
train.head()

,ID,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,y
0,TRAIN_00000,1.006187,-1.962566,1.247535,0.926500,-0.265766,-1.789301,0.470004,-0.139467,0.623996,0.320359,0.078612,83.424500
1,TRAIN_00001,1.024647,-2.472625,1.144386,0.846499,-0.287336,-1.756679,0.503860,-0.219545,0.697607,0.238306,0.081778,79.374109
2,TRAIN_00002,1.062444,-2.451003,1.186546,0.873599,-0.257828,-1.802735,0.498045,-0.194247,0.684134,0.259392,0.095003,82.181616
3,TRAIN_00003,1.089189,-2.458470,1.184531,0.810867,-0.276517,-1.787739,0.503359,-0.201923,0.686394,0.245736,0.091737,83.006586
4,TRAIN_00004,1.023323,-2.133468,1.242266,0.939837,-0.264515,-1.792044,0.470478,-0.142896,0.623778,0.314610,0.078987,83.051434


In [27]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Create histograms for each variable
# fig, axes = plt.subplots(4, 3, figsize=(20, 20))
# axes = axes.ravel()

# for i, col in enumerate(train.columns[1:]):  # Skip the ID column
#     sns.histplot(train[col], ax=axes[i], kde=True)
#     axes[i].set_title(f'Distribution of {col}')

# plt.tight_layout()
# plt.show()

# # Create box plots for each variable
# fig, axes = plt.subplots(4, 3, figsize=(20, 20))
# axes = axes.ravel()

# for i, col in enumerate(train.columns[1:]):  # Skip the ID column
#     sns.boxplot(y=train[col], ax=axes[i])
#     axes[i].set_title(f'Box plot of {col}')

# plt.tight_layout()
# plt.show()

## 2. 데이터 전처리

### 이상치 처리

In [28]:
# #이상치 처리: IQR 방법 사용
# def remove_outliers_iqr(df, column):
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
#     return df

# outlier_columns = ['x_1', 'x_2', 'x_4', 'x_5', 'x_6', 'x_7', 'x_9', 'x_10']
# for col in outlier_columns:
#     train = remove_outliers_iqr(train, col)

### 피쳐 변환

In [29]:
#  피처 변환: 로그 변환 (양수만 변환 가능)
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

train_x = train.drop(columns=['ID', 'y'])
test_x = test.drop(columns=['ID'])
train_y = train['y']

# # 다항 피처 생성 및 변환
# train_x_poly = poly.fit_transform(train_x)
# test_x_poly = poly.transform(test_x)

### 데이터 스케일링

In [30]:
# 3. 데이터 스케일링
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
test_x_scaled = scaler.fit_transform(test_x)

In [31]:
# from sklearn.preprocessing import RobustScaler
# # RobustScaler 초기화
# scaler = RobustScaler()

# # 데이터를 피팅하고 변환
# train_x_scaled = scaler.fit_transform(train_x)
# test_x_scaled = scaler.fit_transform(test_x)

## 3. 분석 모델 설계

In [32]:
# 개별 모델 정의
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=100,random_state=42)

# Voting Regressor로 앙상블 모델 생성
voting_regressor = VotingRegressor(estimators=[('lr', lr), ('rf', rf)])

## 4. 모델 학습

In [33]:
voting_regressor.fit(train_x_scaled, train_y)

VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('rf', RandomForestRegressor(random_state=42))])

## 5. 예측값 생성
학습한 모델을 사용하여 예측값을 생성합니다.

In [34]:
y_pred = voting_regressor.predict(test_x_scaled)

## 5. 제출 파일 생성
submission 파일을 만들어서 제출합니다.

In [35]:
from datetime import datetime

submission = pd.read_csv('sample_submission.csv')

# 현재 날짜와 시간을 'YYYYMMDD_HHMMSS' 형식으로 포맷팅
today_datetime = datetime.today().strftime('%Y%m%d_%H%M')
submission['y'] = y_pred

# 포맷팅한 날짜와 시간으로 파일 이름 생성
submission.to_csv(f'./submission_{today_datetime}.csv', index=False)
